In [1]:
%config Completer.use_jedi = False

In [2]:
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
import random
from utils import *

In [3]:
# import the necessary packages
from tensorflow.keras.models import Model
from tensorflow.keras.layers import *
from tensorflow.keras import datasets
from tensorflow.keras.utils import to_categorical

In [4]:
tf.config.list_physical_devices('GPU')

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

# Data preparation

In [5]:
(train_images, train_labels), (test_images, test_labels) = datasets.cifar10.load_data()

In [6]:
train_images.shape

(50000, 32, 32, 3)

In [7]:
test_images.shape

(10000, 32, 32, 3)

In [8]:
train_images.astype('float32') 
test_images.astype('float32') 

train_images = train_images/255.
test_images = test_images/255.

In [9]:
train_labels_cat = to_categorical(train_labels)
test_labels_cat = to_categorical(test_labels)

# Vanilla network

In [10]:
def build_vanilla_model(shape, class_num):
    # specify the inputs for the feature extractor network
    inputs = Input(shape)

    x = Conv2D(32, (3, 3), activation="relu")(inputs)
    x = MaxPooling2D(pool_size=(2, 2))(x)

    x = Conv2D(64, (2, 2), activation="relu")(x)
    x = MaxPooling2D(pool_size=(2,2))(x)
    x = Dropout(0.2)(x)
    
    x = Conv2D(128, (2, 2), activation="relu")(x)
    x = MaxPooling2D(pool_size=(2,2))(x)
    x = Dropout(0.2)(x)
    
    x = Flatten()(x)

    x = Dense(32, activation='relu')(x)
    x = Dropout(0.2)(x)
    
    outputs = Dense(class_num, activation='softmax')(x)

    # build the model
    model = Model(inputs, outputs)
    
    model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

    return model

### model on the whole dataset

learning

In [11]:
# model = build_vanilla_model(train_images[0].shape, 10)

In [12]:
# history = model.fit(train_images, train_labels_cat, epochs=200, batch_size=512, 
#                     validation_data=(test_images, test_labels_cat))

In [13]:
# print_history_chart(history)

In [14]:
# model.save('vanilla_cifar10.h5')

evaluation

In [15]:
model = tf.keras.models.load_model("vanilla_cifar10.h5")

In [16]:
results_vanilla_full = full_evaluate(model, test_images, test_labels)

accuracy_score: 0.789
f1_score_macro: 0.7843261383766078
f1_score_weighted: 0.7936738616233923


### models on the smaller datasets

#### 50% of the training data

learning

In [46]:
# model = build_vanilla_model(train_images[0].shape, 10)

In [47]:
# train_images_50p = train_images[:len(train_images)//2]
# train_labels_cat_50p = train_labels_cat[:len(train_labels_cat)//2]

In [42]:
# history = model.fit(train_images_50p, train_labels_cat_50p, epochs=300, batch_size=512, 
#                     validation_data=(test_images, test_labels_cat))

In [39]:
# print_history_chart(history)

In [41]:
# model.save('vanilla_50p_cifar10.h5')

evaluation

In [43]:
model = tf.keras.models.load_model("vanilla_50p_cifar10.h5")

In [44]:
results_vanilla_50p = full_evaluate(model, test_images, test_labels)

accuracy_score: 0.7546
f1_score_macro: 0.7541098874413018
f1_score_weighted: 0.7550901125586983


#### 10% of the training data

learning

In [48]:
model = build_vanilla_model(train_images[0].shape, 10)

In [49]:
train_images_10p = train_images[:len(train_images)//10]
train_labels_cat_10p = train_labels_cat[:len(train_labels_cat)//10]

In [50]:
history = model.fit(train_images_10p, train_labels_cat_10p, epochs=400, batch_size=512, 
                    validation_data=(test_images, test_labels_cat))

Epoch 1/400
10/10 [==============================] - 1s 73ms/step - loss: 2.3036 - accuracy: 0.1082 - val_loss: 2.2894 - val_accuracy: 0.1647
Epoch 2/400
10/10 [==============================] - 0s 24ms/step - loss: 2.2592 - accuracy: 0.1598 - val_loss: 2.2174 - val_accuracy: 0.1586
Epoch 3/400
10/10 [==============================] - 0s 22ms/step - loss: 2.1605 - accuracy: 0.1934 - val_loss: 2.0956 - val_accuracy: 0.2221
Epoch 4/400
10/10 [==============================] - 0s 23ms/step - loss: 2.0806 - accuracy: 0.2252 - val_loss: 2.0166 - val_accuracy: 0.2583
Epoch 5/400
10/10 [==============================] - 0s 22ms/step - loss: 2.0031 - accuracy: 0.2602 - val_loss: 1.9338 - val_accuracy: 0.2995
Epoch 6/400
10/10 [==============================] - 0s 23ms/step - loss: 1.9330 - accuracy: 0.2826 - val_loss: 1.8629 - val_accuracy: 0.3298
Epoch 7/400
10/10 [==============================] - 0s 22ms/step - loss: 1.8769 - accuracy: 0.3026 - val_loss: 1.8035 - val_accuracy: 0.3408
Epoch 

10/10 [==============================] - 0s 22ms/step - loss: 1.1526 - accuracy: 0.5856 - val_loss: 1.2305 - val_accuracy: 0.5649
Epoch 59/400
10/10 [==============================] - 0s 22ms/step - loss: 1.1540 - accuracy: 0.5898 - val_loss: 1.2397 - val_accuracy: 0.5623
Epoch 60/400
10/10 [==============================] - 0s 22ms/step - loss: 1.1405 - accuracy: 0.5888 - val_loss: 1.2227 - val_accuracy: 0.5664
Epoch 61/400
10/10 [==============================] - 0s 21ms/step - loss: 1.1391 - accuracy: 0.5860 - val_loss: 1.2287 - val_accuracy: 0.5631
Epoch 62/400
10/10 [==============================] - 0s 22ms/step - loss: 1.1137 - accuracy: 0.6036 - val_loss: 1.2223 - val_accuracy: 0.5654
Epoch 63/400
10/10 [==============================] - 0s 24ms/step - loss: 1.1045 - accuracy: 0.6054 - val_loss: 1.2120 - val_accuracy: 0.5704
Epoch 64/400
10/10 [==============================] - 0s 24ms/step - loss: 1.1091 - accuracy: 0.6098 - val_loss: 1.2100 - val_accuracy: 0.5758
Epoch 65/400

10/10 [==============================] - 0s 21ms/step - loss: 0.8093 - accuracy: 0.7124 - val_loss: 1.1408 - val_accuracy: 0.5996
Epoch 116/400
10/10 [==============================] - 0s 21ms/step - loss: 0.8095 - accuracy: 0.7072 - val_loss: 1.1404 - val_accuracy: 0.6041
Epoch 117/400
10/10 [==============================] - 0s 24ms/step - loss: 0.8011 - accuracy: 0.7162 - val_loss: 1.1288 - val_accuracy: 0.6043
Epoch 118/400
10/10 [==============================] - 0s 22ms/step - loss: 0.7796 - accuracy: 0.7170 - val_loss: 1.1261 - val_accuracy: 0.6103
Epoch 119/400
10/10 [==============================] - 0s 25ms/step - loss: 0.8040 - accuracy: 0.7018 - val_loss: 1.1524 - val_accuracy: 0.6047
Epoch 120/400
10/10 [==============================] - 0s 24ms/step - loss: 0.7924 - accuracy: 0.7156 - val_loss: 1.1350 - val_accuracy: 0.6036
Epoch 121/400
10/10 [==============================] - 0s 24ms/step - loss: 0.7852 - accuracy: 0.7228 - val_loss: 1.1303 - val_accuracy: 0.6061
Epoch 

Epoch 172/400
10/10 [==============================] - 0s 25ms/step - loss: 0.5949 - accuracy: 0.7772 - val_loss: 1.1666 - val_accuracy: 0.6189
Epoch 173/400
10/10 [==============================] - 0s 26ms/step - loss: 0.5949 - accuracy: 0.7896 - val_loss: 1.1587 - val_accuracy: 0.6219
Epoch 174/400
10/10 [==============================] - 0s 25ms/step - loss: 0.5693 - accuracy: 0.8006 - val_loss: 1.1562 - val_accuracy: 0.6188
Epoch 175/400
10/10 [==============================] - 0s 24ms/step - loss: 0.5797 - accuracy: 0.7880 - val_loss: 1.1507 - val_accuracy: 0.6198
Epoch 176/400
10/10 [==============================] - 0s 25ms/step - loss: 0.5625 - accuracy: 0.7924 - val_loss: 1.1797 - val_accuracy: 0.6179
Epoch 177/400
10/10 [==============================] - 0s 24ms/step - loss: 0.5631 - accuracy: 0.7938 - val_loss: 1.1701 - val_accuracy: 0.6210
Epoch 178/400
10/10 [==============================] - 0s 22ms/step - loss: 0.5675 - accuracy: 0.7940 - val_loss: 1.1542 - val_accuracy:

Epoch 229/400
10/10 [==============================] - 0s 24ms/step - loss: 0.4395 - accuracy: 0.8398 - val_loss: 1.2313 - val_accuracy: 0.6246
Epoch 230/400
10/10 [==============================] - 0s 24ms/step - loss: 0.4453 - accuracy: 0.8382 - val_loss: 1.2415 - val_accuracy: 0.6218
Epoch 231/400
10/10 [==============================] - 0s 25ms/step - loss: 0.4463 - accuracy: 0.8406 - val_loss: 1.2301 - val_accuracy: 0.6262
Epoch 232/400
10/10 [==============================] - 0s 22ms/step - loss: 0.4377 - accuracy: 0.8406 - val_loss: 1.2244 - val_accuracy: 0.6251
Epoch 233/400
10/10 [==============================] - 0s 25ms/step - loss: 0.4293 - accuracy: 0.8408 - val_loss: 1.2334 - val_accuracy: 0.6296
Epoch 234/400
10/10 [==============================] - 0s 24ms/step - loss: 0.4221 - accuracy: 0.8468 - val_loss: 1.2168 - val_accuracy: 0.6270
Epoch 235/400
10/10 [==============================] - 0s 22ms/step - loss: 0.4301 - accuracy: 0.8486 - val_loss: 1.2415 - val_accuracy:

Epoch 286/400
10/10 [==============================] - 0s 24ms/step - loss: 0.3313 - accuracy: 0.8814 - val_loss: 1.3303 - val_accuracy: 0.6263
Epoch 287/400
10/10 [==============================] - 0s 22ms/step - loss: 0.3577 - accuracy: 0.8674 - val_loss: 1.3271 - val_accuracy: 0.6262
Epoch 288/400
10/10 [==============================] - 0s 22ms/step - loss: 0.3702 - accuracy: 0.8658 - val_loss: 1.2961 - val_accuracy: 0.6261
Epoch 289/400
10/10 [==============================] - 0s 22ms/step - loss: 0.3636 - accuracy: 0.8720 - val_loss: 1.2993 - val_accuracy: 0.6273
Epoch 290/400
10/10 [==============================] - 0s 22ms/step - loss: 0.3372 - accuracy: 0.8786 - val_loss: 1.3105 - val_accuracy: 0.6262
Epoch 291/400
10/10 [==============================] - 0s 22ms/step - loss: 0.3392 - accuracy: 0.8794 - val_loss: 1.3342 - val_accuracy: 0.6303
Epoch 292/400
10/10 [==============================] - 0s 22ms/step - loss: 0.3383 - accuracy: 0.8842 - val_loss: 1.3254 - val_accuracy:

Epoch 343/400
10/10 [==============================] - 0s 23ms/step - loss: 0.2943 - accuracy: 0.8936 - val_loss: 1.3785 - val_accuracy: 0.6312
Epoch 344/400
10/10 [==============================] - 0s 23ms/step - loss: 0.3033 - accuracy: 0.8888 - val_loss: 1.3886 - val_accuracy: 0.6285
Epoch 345/400
10/10 [==============================] - 0s 25ms/step - loss: 0.2933 - accuracy: 0.8930 - val_loss: 1.4013 - val_accuracy: 0.6306
Epoch 346/400
10/10 [==============================] - 0s 25ms/step - loss: 0.2980 - accuracy: 0.8968 - val_loss: 1.3661 - val_accuracy: 0.6299
Epoch 347/400
10/10 [==============================] - 0s 24ms/step - loss: 0.2980 - accuracy: 0.8970 - val_loss: 1.3887 - val_accuracy: 0.6299
Epoch 348/400
10/10 [==============================] - 0s 21ms/step - loss: 0.3023 - accuracy: 0.8964 - val_loss: 1.3709 - val_accuracy: 0.6302
Epoch 349/400
10/10 [==============================] - 0s 23ms/step - loss: 0.2924 - accuracy: 0.8976 - val_loss: 1.3893 - val_accuracy:

Epoch 400/400
10/10 [==============================] - 0s 24ms/step - loss: 0.2619 - accuracy: 0.9086 - val_loss: 1.4379 - val_accuracy: 0.6355


In [52]:
# print_history_chart(history)

In [54]:
# model.save('vanilla_10p_cifar10.h5')

evaluation

In [55]:
model = tf.keras.models.load_model("vanilla_10p_cifar10.h5")

In [56]:
results_vanilla_10p = full_evaluate(model, test_images, test_labels)

accuracy_score: 0.6355
f1_score_macro: 0.6314960493654187
f1_score_weighted: 0.6395039506345813


# Siemese network

In [11]:
def build_siamese_vanilla_model(shape, class_num):
    inputs1 = Input(shape)
    inputs2 = Input(shape)
    
    def one_side(inputs):
        x = Conv2D(32, (3, 3), activation="relu")(inputs)
        x = MaxPooling2D(pool_size=(2, 2))(x)

        x = Conv2D(64, (2, 2), activation="relu")(x)
        x = MaxPooling2D(pool_size=(2,2))(x)
        x = Dropout(0.2)(x)

        x = Conv2D(128, (2, 2), activation="relu")(x)
        x = MaxPooling2D(pool_size=(2,2))(x)
        x = Dropout(0.2)(x)

        x = Flatten()(x)

        x = Dense(32, activation='relu')(x)
        x = Dropout(0.2)(x)
        
        return x
    
    x1 = one_side(inputs1)
    x2 = one_side(inputs2)
    
    x = concatenate([x1, x2])
    
    outputs = Dense(class_num, activation='softmax')(x)

    # build the model
    model = Model([inputs1, inputs2], outputs)
    
    model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

    return model

In [12]:
def make_pairs(labels):
    samples = {a: list(filter(lambda x: True if x[1]==a else False, enumerate(labels))) for a in np.unique(labels)}
    labels2 = []
    for l in labels:
        labels2.append(random.choice(samples[l[0]])[0])
    return labels2

### model on the whole datasets

learning

In [13]:
# model = build_siamese_vanilla_model(train_images[0].shape, 10)

In [14]:
# idx2 = make_pairs(train_labels)

# y = np.concatenate([train_labels,train_labels[idx2]])
# X1 = np.concatenate([train_images,train_images])
# X2 = np.concatenate([train_images,train_images[idx2]])

# y = to_categorical(y)

In [15]:
# history = model.fit([X1, X2], y, epochs=200, batch_size=512, shuffle=True,
#          validation_data=([test_images, test_images], test_labels_cat))


In [16]:
# print_history_chart(history)

In [17]:
# model.save('siamese_vanilla_cifar10.h5')

evaluate

In [25]:
model = tf.keras.models.load_model("siamese_vanilla_cifar10.h5")


In [26]:
results_siamese_vanilla = full_evaluate(model, [test_images, test_images], test_labels)

accuracy_score: 0.8135
f1_score_macro: 0.8118478425773764
f1_score_weighted: 0.8151521574226235


### models on the smaller datasets

#### 50% of the taining data

learning

In [13]:
model = build_siamese_vanilla_model(train_images[0].shape, 10)

In [14]:
train_images_50p = train_images[:len(train_images)//2]
train_labels_50p = train_labels[:len(train_labels)//2]

In [21]:
# idx2 = make_pairs(train_labels_50p)
# idx3 = make_pairs(train_labels_50p)

# y = np.concatenate([train_labels_50p, train_labels_50p[idx2], train_labels_50p[idx3]])
# X1 = np.concatenate([train_images_50p, train_images_50p, train_images_50p])
# X2 = np.concatenate([train_images_50p, train_images_50p[idx2], train_images_50p[idx3]])
# y = to_categorical(y)

In [20]:
# history = model.fit([X1, X2], y, epochs=300, batch_size=512, shuffle=True,
#          validation_data=([test_images, test_images], test_labels_cat))

In [18]:
# model.save('siamese_vanilla_50p_cifar10.h5')

evaluate

In [22]:
model = tf.keras.models.load_model("siamese_vanilla_50p_cifar10.h5")

In [24]:
results_siamese_vanilla_50p = full_evaluate(model, [test_images, test_images], test_labels)

accuracy_score: 0.7755
f1_score_macro: 0.7745039812753725
f1_score_weighted: 0.7764960187246276


#### 10% of the taining data

learning

In [18]:
# model = build_siamese_vanilla_model(train_images[0].shape, 10)

In [19]:
# train_images_10p = train_images[:len(train_images)//10]
# train_labels_10p = train_labels[:len(train_labels)//10]

In [20]:
# idx2 = make_pairs(train_labels_10p)
# idx3 = make_pairs(train_labels_10p)
# idx4 = make_pairs(train_labels_10p)
# idx5 = make_pairs(train_labels_10p)

# y = np.concatenate([train_labels_10p, train_labels_10p[idx2], train_labels_10p[idx3], train_labels_10p[idx4],
#                    train_labels_10p[idx5]])
# X1 = np.concatenate([train_images_10p, train_images_10p, train_images_10p, train_images_10p, train_images_10p])
# X2 = np.concatenate([train_images_10p, train_images_10p[idx2], train_images_10p[idx3], train_images_10p[idx4],
#                     train_images_10p[idx5]])
# y = to_categorical(y)

In [21]:
# history = model.fit([X1, X2], y, epochs=300, batch_size=512, shuffle=True,
#          validation_data=([test_images, test_images], test_labels_cat))

In [22]:
# model.save('siamese_vanilla_10p_cifar10.h5')

evaluate

In [23]:
model = tf.keras.models.load_model("siamese_vanilla_10p_cifar10.h5")

In [24]:
results_siamese_vanilla_10p = full_evaluate(model, [test_images, test_images], test_labels)

accuracy_score: 0.6445
f1_score_macro: 0.6421815577943102
f1_score_weighted: 0.6468184422056897
